# Create a Neural Network having only Dense layers to perform multiclass classification of X-ray Images of Chest X-rays, using functional API and train it on NIH Chest X-ray Dataset using custom training loop.

# Let's download our Dataset from kaggle

# We will be using `Random Sample of NIH Chest X-ray Dataset`

https://www.kaggle.com/datasets/nih-chest-xrays/sample

In [ ]:
from google.colab import files

In [ ]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"navneet14","key":"a8d8b713c16b1d51f09f2ee56e2b3d59"}'}

In [ ]:
!mkdir ~/.kaggle

In [ ]:
!mv /content/kaggle.json ~/.kaggle

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!pip install kaggle

In [1]:
!kaggle datasets download -d nih-chest-xrays/sample

Dataset URL: https://www.kaggle.com/datasets/nih-chest-xrays/sample
License(s): CC0-1.0
100% 4.18G/4.20G [01:23<00:00, 133MB/s]
100% 4.20G/4.20G [01:23<00:00, 54.2MB/s]


In [2]:
!unzip /content/sample.zip

Streaming output truncated to the last 5000 lines.
  inflating: sample/sample/images/00003124_000.png  
  inflating: sample/sample/images/00003129_000.png  
  inflating: sample/sample/images/00003129_001.png  
  inflating: sample/sample/images/00003129_004.png  
  inflating: sample/sample/images/00003129_009.png  
  inflating: sample/sample/images/00003129_011.png  
  inflating: sample/sample/images/00003135_000.png  
  inflating: sample/sample/images/00003140_007.png  
  inflating: sample/sample/images/00003159_001.png  
  inflating: sample/sample/images/00003169_001.png  
  inflating: sample/sample/images/00003177_002.png  
  inflating: sample/sample/images/00003186_004.png  
  inflating: sample/sample/images/00003186_005.png  
  inflating: sample/sample/images/00003190_018.png  
  inflating: sample/sample/images/00003191_001.png  
  inflating: sample/sample/images/00003198_002.png  
  inflating: sample/sample/images/00003198_005.png  
  inflating: sample/sample/images/00003198_009.p

# importing required libraries

In [376]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten
import tensorflow as tf
from keras.optimizers import RMSprop

# Read the Dataset

In [377]:
# Loading  the dataset
df =pd.read_csv("/content/sample_labels.csv")

# shape of the dataframe

In [378]:
df.shape

(5606, 11)

#Let's take a preliminary look on our dataset

In [379]:
df.head()

Image Index                                     Finding Labels  \
0  00000013_005.png  Emphysema|Infiltration|Pleural_Thickening|Pneu...   
1  00000013_026.png                             Cardiomegaly|Emphysema   
2  00000017_001.png                                         No Finding   
3  00000030_001.png                                        Atelectasis   
4  00000032_001.png                        Cardiomegaly|Edema|Effusion   

   Follow-up #  Patient ID Patient Age Patient Gender View Position  \
0            5          13        060Y              M            AP   
1           26          13        057Y              M            AP   
2            1          17        077Y              M            AP   
3            1          30        079Y              M            PA   
4            1          32        055Y              F            AP   

   OriginalImageWidth  OriginalImageHeight  OriginalImagePixelSpacing_x  \
0                3056                 2544                        0.139   
1                2500                 2048                        0.168   
2                2500                 2048                        0.168   
3                2992                 2991                        0.143   
4                2500                 2048                        0.168   

   OriginalImagePixelSpacing_y  
0                        0.139  
1                        0.168  
2                        0.168  
3                        0.143  
4                        0.168

# View summary of our dataset using `info` method in pandas

In [380]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5606 entries, 0 to 5605
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Image Index                  5606 non-null   object 
 1   Finding Labels               5606 non-null   object 
 2   Follow-up #                  5606 non-null   int64  
 3   Patient ID                   5606 non-null   int64  
 4   Patient Age                  5606 non-null   object 
 5   Patient Gender               5606 non-null   object 
 6   View Position                5606 non-null   object 
 7   OriginalImageWidth           5606 non-null   int64  
 8   OriginalImageHeight          5606 non-null   int64  
 9   OriginalImagePixelSpacing_x  5606 non-null   float64
 10  OriginalImagePixelSpacing_y  5606 non-null   float64
dtypes: float64(2), int64(4), object(5)
memory usage: 481.9+ KB


# Checking for any null values

In [381]:
df.isnull().sum()

Image Index                    0
Finding Labels                 0
Follow-up #                    0
Patient ID                     0
Patient Age                    0
Patient Gender                 0
View Position                  0
OriginalImageWidth             0
OriginalImageHeight            0
OriginalImagePixelSpacing_x    0
OriginalImagePixelSpacing_y    0
dtype: int64

# As we can see we there is `0` null values in every coloumn so we can go forward without any preprocessing of our dataset

# Let's see features included in our dataset

In [382]:
df.columns

Index(['Image Index', 'Finding Labels', 'Follow-up #', 'Patient ID',
       'Patient Age', 'Patient Gender', 'View Position', 'OriginalImageWidth',
       'OriginalImageHeight', 'OriginalImagePixelSpacing_x',
       'OriginalImagePixelSpacing_y'],
      dtype='object')

# counting the number of images

In [383]:
len(os.listdir("/content/sample/images"))

5606

#let's load a random image

In [384]:
img=cv2.imread("/content/sample/images/00000013_005.png")

In [385]:
import matplotlib.pyplot as plt

In [386]:
plt.imshow(img)

# Shape of the image


In [387]:
img.shape

(1024, 1024, 3)

- Width: 1024
- Height: 1024
- Channels: 3(RGB)



# Filtering Necessary Columns

In [388]:
columns = ['Image Index', 'Finding Labels']
df = df[columns]


In [389]:
no_finding_count = df[df['Finding Labels'] == 'No Finding']

In [390]:
findings = df[df['Finding Labels'] != 'No Finding']


In [391]:
findings.shape[0]

2562

In [392]:
no_finding_count

Image Index Finding Labels
2     00000017_001.png     No Finding
6     00000042_002.png     No Finding
7     00000057_001.png     No Finding
9     00000061_019.png     No Finding
12    00000080_005.png     No Finding
...                ...            ...
5597  00030639_001.png     No Finding
5600  00030704_000.png     No Finding
5601  00030712_000.png     No Finding
5604  00030792_000.png     No Finding
5605  00030797_000.png     No Finding

[3044 rows x 2 columns]

In [393]:
no_finding_count.shape

(3044, 2)

In [394]:
no_finding_count.shape[0]/df.shape[0]

0.5429896539422048

#Out of 5606 we had 3044 `No Finding`  which is almost 54% it signifies our dataset is almost a `balanced` but let  make it `fully balanced`

In [395]:
from sklearn.utils import resample

In [396]:
no_finding_downsampled = resample(no_finding_count, replace=False,n_samples=len(findings),random_state=42)

In [397]:
no_finding_downsampled.shape[0]

2562

#Let's concat both `findings` and `no_finding_downsampled`

In [398]:
df_balanced = pd.concat([no_finding_downsampled, findings])

In [399]:
df_balanced.shape[0]

5124


# Process labels to create a mapping

In [400]:

labels = set()
for label_list in df_balanced['Finding Labels'].str.split('|'):
    labels.update(label_list)


# Printing the labels

In [401]:
labels

{'Atelectasis',
 'Cardiomegaly',
 'Consolidation',
 'Edema',
 'Effusion',
 'Emphysema',
 'Fibrosis',
 'Hernia',
 'Infiltration',
 'Mass',
 'No Finding',
 'Nodule',
 'Pleural_Thickening',
 'Pneumonia',
 'Pneumothorax'}

# Creating map of all labels with index

In [402]:
labels = sorted(labels)
label_map = {label: idx for idx, label in enumerate(labels)}


In [403]:
all_labels=labels

In [404]:
label_map


{'Atelectasis': 0,
 'Cardiomegaly': 1,
 'Consolidation': 2,
 'Edema': 3,
 'Effusion': 4,
 'Emphysema': 5,
 'Fibrosis': 6,
 'Hernia': 7,
 'Infiltration': 8,
 'Mass': 9,
 'No Finding': 10,
 'Nodule': 11,
 'Pleural_Thickening': 12,
 'Pneumonia': 13,
 'Pneumothorax': 14}

In [405]:
len(labels)

15

# One Hot Encoding of the Labels

# Initialize an empty array for the one-hot encoded labels

In [406]:

one_hot_labels = np.zeros((len(df), len(labels)), dtype=int)


In [407]:
one_hot_labels

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

# one hot encoding

In [408]:

for i, label_str in enumerate(df_balanced['Finding Labels']):
    labels = label_str.split('|')  # Ensure labels are split by '|'
    for label in labels:
        index = label_map.get(label)
        if index is not None:
            one_hot_labels[i, index] = 1
        else:
            print(f"Label '{label}' not found in label_map.")


In [409]:
one_hot_labels

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

# Add the one-hot encoded labels to the DataFrame

In [410]:

one_hot_df = pd.DataFrame(one_hot_labels, columns=all_labels)
df_balanced = df_balanced.join(one_hot_df)

In [411]:
df_balanced.head(20)

Image Index Finding Labels  Atelectasis  Cardiomegaly  \
524   00002714_001.png     No Finding            0             0   
5095  00027075_001.png     No Finding            0             0   
248   00001303_000.png     No Finding            0             0   
926   00004858_052.png     No Finding            0             0   
1632  00008480_000.png     No Finding            0             0   
3726  00018259_004.png     No Finding            0             0   
3132  00015442_026.png     No Finding            1             0   
4504  00022458_013.png     No Finding            0             0   
4119  00020290_009.png     No Finding            0             0   
1271  00006691_000.png     No Finding            0             0   
3518  00017333_002.png     No Finding            0             0   
4222  00020736_000.png     No Finding            0             0   
2872  00014201_000.png     No Finding            0             0   
122   00000662_001.png     No Finding            0             0   
1554  00008051_014.png     No Finding            0             0   
792   00004144_002.png     No Finding            0             0   
3071  00015133_002.png     No Finding            0             0   
2641  00013261_009.png     No Finding            0             0   
1885  00009912_000.png     No Finding            0             0   
3410  00016799_000.png     No Finding            0             0   

      Consolidation  Edema  Effusion  Emphysema  Fibrosis  Hernia  \
524               0      0         0          0         0       0   
5095              0      0         0          0         0       0   
248               0      0         0          0         0       0   
926               0      0         0          0         0       0   
1632              0      0         0          0         0       0   
3726              0      0         1          0         0       0   
3132              1      0         0          0         0       0   
4504              0      0         0          0         0       0   
4119              0      0         0          0         0       0   
1271              0      0         0          0         0       0   
3518              0      0         0          0         0       0   
4222              0      0         0          0         0       0   
2872              0      0         1          0         0       0   
122               0      0         0          0         0       0   
1554              0      0         0          0         0       0   
792               0      0         0          0         0       0   
3071              0      1         0          0         0       0   
2641              0      0         0          0         0       0   
1885              0      0         0          0         0       0   
3410              0      0         0          0         0       0   

      Infiltration  Mass  No Finding  Nodule  Pleural_Thickening  Pneumonia  \
524              0     0           1       0                   0          0   
5095             0     0           0       0                   0          0   
248              0     0           1       0                   0          0   
926              0     0           1       0                   0          0   
1632             0     0           1       0                   0          0   
3726             0     0           0       0                   0          0   
3132             0     0           0       0                   0          0   
4504             1     0           0       0                   0          0   
4119             1     0           0       0                   0          0   
1271             0     0           1       0                   0          0   
3518             1     0           0       0                   0          0   
4222             1     0           0       0                   0          0   
2872             0     0           0       0                   0          0   
122              0     0  

# Split the dataset into training and testing

In [412]:

train_df, test_df = train_test_split(df_balanced, test_size=0.2, random_state=42)

In [413]:
train_df.head()

Image Index Finding Labels  Atelectasis  Cardiomegaly  \
1245  00006535_002.png     No Finding            0             0   
2132  00010992_009.png       Effusion            0             0   
2475  00012437_000.png       Fibrosis            0             0   
3848  00018944_012.png    Atelectasis            0             0   
4823  00025368_010.png     No Finding            0             0   

      Consolidation  Edema  Effusion  Emphysema  Fibrosis  Hernia  \
1245              0      0         0          0         0       0   
2132              0      0         0          0         0       0   
2475              0      0         0          0         0       0   
3848              0      0         0          0         0       0   
4823              0      0         0          0         0       0   

      Infiltration  Mass  No Finding  Nodule  Pleural_Thickening  Pneumonia  \
1245             0     0           1       0                   0          0   
2132             0     0           1       0                   0          0   
2475             0     0           1       0                   0          0   
3848             0     0           0       0                   1          0   
4823             1     0           0       0                   0          0   

      Pneumothorax  
1245             0  
2132             0  
2475             0  
3848             0  
4823             0

# Image Preprocessing function and Normalizing

In [414]:

def preprocess_image(image_path, target_size=(128, 128)):
    image = cv2.imread(image_path)
    image = cv2.resize(image, target_size)
    image = image / 255.0  # Normalize image
    return image

/content/sample/images/00000030_001.png

# Custom data generator function

In [415]:

def training_data_generator(data_df, data_dir, batch_size=32, image_size=(128, 128)):
    labels = data_df.columns[2:]
    while True:
        for i in range(len(data_df) // batch_size):
            batch_df = data_df.iloc[i * batch_size: (i + 1) * batch_size]
            X = np.empty((batch_size, *image_size, 3), dtype=np.float32)
            Y = np.empty((batch_size, len(labels)), dtype=np.float32)

            for j, (_, row) in enumerate(batch_df.iterrows()):
                img_name = row['Image Index']
                img_path = os.path.join(data_dir, img_name)
                X[j,] = preprocess_image(img_path, target_size=image_size)
                Y[j] = row[labels].values

            yield X, Y

# directory where images are available

In [416]:

data_dir="/content/sample/images/"

In [417]:
datagen = training_data_generator(train_df,data_dir, batch_size=32, image_size=(128, 128))

In [418]:
X_train_mb, Y_train_mb = datagen.__next__()

In [419]:
X_train_mb

array([[[[0.43137255, 0.43137255, 0.43137255],
         [0.24705882, 0.24705882, 0.24705882],
         [0.15686275, 0.15686275, 0.15686275],
         ...,
         [0.14509805, 0.14509805, 0.14509805],
         [0.14901961, 0.14901961, 0.14901961],
         [0.14901961, 0.14901961, 0.14901961]],

        [[0.40784314, 0.40784314, 0.40784314],
         [0.21176471, 0.21176471, 0.21176471],
         [0.14901961, 0.14901961, 0.14901961],
         ...,
         [0.14117648, 0.14117648, 0.14117648],
         [0.14509805, 0.14509805, 0.14509805],
         [0.14901961, 0.14901961, 0.14901961]],

        [[0.3882353 , 0.3882353 , 0.3882353 ],
         [0.21176471, 0.21176471, 0.21176471],
         [0.14901961, 0.14901961, 0.14901961],
         ...,
         [0.14117648, 0.14117648, 0.14117648],
         [0.14901961, 0.14901961, 0.14901961],
         [0.14509805, 0.14509805, 0.14509805]],

        ...,

        [[0.16470589, 0.16470589, 0.16470589],
         [0.14901961, 0.14901961, 0.14901961]

# Multi-Layer Perceptron (MLP) Architecture Definition

In [420]:
def functional_mlp(input_shape, num_classes):
    inp = Input(shape=input_shape)
    x = Flatten()(inp)  # flatten the input image
    x = Dense(units=1024, activation="relu")(x)
    x = Dense(units=512, activation="relu")(x)
    x = Dense(units=256, activation="relu")(x)
    x = Dense(units=128, activation="relu")(x)
    x = Dense(units=64, activation="relu")(x)
    nn_out = Dense(units=num_classes, activation="sigmoid")(x)


    return Model(inputs=[inp], outputs=[nn_out])


In [421]:
len(all_labels)

15

In [422]:
input_shape = (128, 128, 3)
num_classes = len(all_labels)
functional_nn = functional_mlp(input_shape, num_classes)


In [423]:
functional_nn.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 flatten_7 (Flatten)         (None, 49152)             0         
                                                                 
 dense_42 (Dense)            (None, 1024)              50332672  
                                                                 
 dense_43 (Dense)            (None, 512)               524800    
                                                                 
 dense_44 (Dense)            (None, 256)               131328    
                                                                 
 dense_45 (Dense)            (None, 128)               32896     
                                                                 
 dense_46 (Dense)            (None, 64)                8256

In [424]:
def binary_cross_entropy_loss(Y_hat,Y_true):

    return tf.reduce_mean(tf.keras.losses.binary_crossentropy(y_true=Y_true,y_pred=Y_hat))

In [425]:
optimizer = RMSprop()

In [426]:
labels=all_labels

# Custom training loop

In [427]:

epochs = 10
batch_size = 32
image_size = (128, 128)
data_dir = "/content/sample/images/"
steps_per_epoch = len(train_df) // batch_size
prev_loss = float('inf')
tol=0.1;
previous_epoch_loss = float('inf')

In [428]:
from tensorflow.keras.metrics import BinaryAccuracy

# it includes computation of gradients and updating weights also calculation of forward and backward passes

In [429]:
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}")
    total_loss = 0
    total_accuracy = 0

    for step in range(steps_per_epoch):
        X_train_mb, Y_train_mb = next(training_data_generator(train_df, data_dir, batch_size=batch_size, image_size=image_size))

        with tf.GradientTape() as tape:
            Y_pred = functional_nn(X_train_mb, training=True)
            loss_func = binary_cross_entropy_loss(Y_pred, Y_train_mb)

        gradients = tape.gradient(loss_func, functional_nn.trainable_weights)
        optimizer.apply_gradients(zip(gradients, functional_nn.trainable_weights))

        accuracy_metric.update_state(Y_train_mb, Y_pred)
        batch_accuracy = accuracy_metric.result().numpy()

        total_loss += loss_func.numpy()
        total_accuracy += batch_accuracy


        print(f"Step {step + 1}/{steps_per_epoch} - loss: {loss_func:.4f} - accuracy: {batch_accuracy:.4f}")


    # calculating average loss and accuracy for the epoch
    avg_loss = total_loss / steps_per_epoch
    avg_accuracy = total_accuracy / steps_per_epoch

    print(f"Epoch {epoch + 1} - loss: {avg_loss:.4f} - accuracy: {avg_accuracy:.4f}")
    #check for convergence
    if (previous_epoch_loss - avg_loss) < tol:
            break

    previous_epoch_loss = avg_loss


    # eset the accuracy metric at the end of the epoch
    accuracy_metric.reset_states()


Epoch 1
Step 1/128 - loss: 0.6618 - accuracy: 0.9048
Step 2/128 - loss: 4.3184 - accuracy: 0.8995
Step 3/128 - loss: 24.5785 - accuracy: 0.8875
Step 4/128 - loss: 3.9544 - accuracy: 0.8832
Step 5/128 - loss: 2.5372 - accuracy: 0.8786
Step 6/128 - loss: 2.1576 - accuracy: 0.8806
Step 7/128 - loss: 2.3257 - accuracy: 0.8777
Step 8/128 - loss: 1.7085 - accuracy: 0.8772
Step 9/128 - loss: 0.8078 - accuracy: 0.8790
Step 10/128 - loss: 1.2559 - accuracy: 0.8797
Step 11/128 - loss: 0.6920 - accuracy: 0.8813
Step 12/128 - loss: 0.3300 - accuracy: 0.8827
Step 13/128 - loss: 0.5139 - accuracy: 0.8841
Step 14/128 - loss: 0.5580 - accuracy: 0.8845
Step 15/128 - loss: 0.4674 - accuracy: 0.8858
Step 16/128 - loss: 0.2703 - accuracy: 0.8870
Step 17/128 - loss: 0.2276 - accuracy: 0.8880
Step 18/128 - loss: 0.2104 - accuracy: 0.8891
Step 19/128 - loss: 0.1976 - accuracy: 0.8901
Step 20/128 - loss: 0.1927 - accuracy: 0.8911
Step 21/128 - loss: 0.1828 - accuracy: 0.8921
Step 22/128 - loss: 0.1899 - accur